In [1]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tabulate import tabulate
import yaml

In [2]:
#values for csv importing
csv_opts = {'sep': '|',
           'quotechar': '"',
           #'compression': 'gzip',
           'encoding': 'utf-8'}

In [3]:
with open('arrest_dtypes.yml', 'r') as yamlfile:
        arrest_dtypes = yaml.load(yamlfile, Loader=yaml.FullLoader)

In [4]:
#reading in CSV file
arrests = pd.read_csv('../../data/arrests.csv', **csv_opts, dtype=arrest_dtypes)

In [5]:
#putting event date into m/d/y format
arrests['apprehension_date'] = pd.to_datetime(
    arrests['apprehension_date'], format='%m/%d/%Y')

In [6]:
# adding year and month columns
arrests["year"] = arrests["apprehension_date"].dt.strftime('%Y')
arrests["month"] = arrests["apprehension_date"].dt.strftime('%m')

In [7]:
# counting arrests by fiscal year
arrests.set_index('apprehension_date').groupby(
    pd.Grouper(freq='AS-OCT'))['id'].count()

apprehension_date
2015-10-01    110104
2016-10-01    143470
2017-10-01    158581
2018-10-01    131904
Freq: AS-OCT, Name: id, dtype: int64

In [8]:
# counting arrests by month
arrests.set_index('apprehension_date').groupby(
    pd.Grouper(freq='MS'))['id'].count()

apprehension_date
2015-10-01    10242
2015-11-01     8544
2015-12-01     8778
2016-01-01     8046
2016-02-01     8756
2016-03-01     9669
2016-04-01     9457
2016-05-01     9652
2016-06-01     9103
2016-07-01     8436
2016-08-01    10115
2016-09-01     9306
2016-10-01     9397
2016-11-01     9015
2016-12-01     8998
2017-01-01     9575
2017-02-01    11554
2017-03-01    14084
2017-04-01    12372
2017-05-01    13527
2017-06-01    13972
2017-07-01    13567
2017-08-01    14364
2017-09-01    13045
2017-10-01    14006
2017-11-01    13177
2017-12-01    12307
2018-01-01    13167
2018-02-01    13236
2018-03-01    13737
2018-04-01    13324
2018-05-01    14280
2018-06-01    13315
2018-07-01    12578
2018-08-01    13718
2018-09-01    11736
2018-10-01    12378
2018-11-01    11693
2018-12-01    11216
2019-01-01    12491
2019-02-01    12381
2019-03-01    12136
2019-04-01    12151
2019-05-01    13073
2019-06-01    11208
2019-07-01    11714
2019-08-01    11459
2019-09-01        4
Freq: MS, Name: id, dt

In [9]:
# creating subset of arrests by AOR over time
arrests_by_fy = arrests[["aor", "apprehension_date", "id"]]
arrests_by_fy = arrests_by_fy.groupby(
    ["aor", "apprehension_date"], as_index=False)['id'].count()

# pivot table so dates are rows and AORs are columns
arrests_by_fy = arrests_by_fy.pivot(
    index='apprehension_date', columns='aor', values='id')

# grouping by fiscal year
arrests_by_fy = arrests_by_fy.groupby(
    pd.Grouper(freq='AS-OCT'), dropna=False).sum()


In [26]:
arrests_by_fy.to_csv("arrests_by_fy.csv")

In [28]:
df = pd.read_csv("arrests_by_fy.csv")

In [29]:
df

,apprehension_date,ATL,BAL,BOS,BUF,CHI,DAL,DEN,DET,ELP,...,NYC,PHI,PHO,SEA,SFR,SLC,SNA,SND,SPM,WAS
0,2015-10-01,8866,1239,1858,1173,7055,9634,2284,2241,1611,...,1847,3672,5370,2698,6651,4638,8425,3683,2500,2883
1,2016-10-01,13551,1666,2834,1494,8604,16520,2746,3409,1892,...,2576,4938,6457,3376,7231,5177,8510,4551,4175,4163
2,2017-10-01,15189,1703,2908,1582,9241,17644,2755,3604,2159,...,3476,5143,7162,3067,6210,5754,10749,6000,4658,4359
3,2018-10-01,12268,1330,2266,1206,7769,15571,2215,3094,2103,...,2281,3752,5420,2328,4724,5014,10968,2107,4005,3734


In [10]:
import plotly.express as px

In [24]:
aor

['ATL',
 'BAL',
 'BOS',
 'BUF',
 'CHI',
 'DAL',
 'DEN',
 'DET',
 'ELP',
 'HOU',
 'HQ',
 'LOS',
 'MIA',
 'NEW',
 'NOL',
 'NYC',
 'PHI',
 'PHO',
 'SEA',
 'SFR',
 'SLC',
 'SNA',
 'SND',
 'SPM',
 'WAS']

In [11]:
fy = ['2015-10-01', '2016-10-01', '2017-10-01', '2018-10-01']
aor = arrests_by_fy.columns.categories.tolist()



In [25]:
aor = ['ATL',
 'BAL',
 'BOS',
 'BUF',
 'CHI',
 'DAL',
 'DEN',
 'DET',
 'ELP',
 'HOU',
 'HQ',
 'LOS',
 'MIA',
 'NEW',
 'NOL',
 'NYC',
 'PHI',
 'PHO',
 'SEA',
 'SFR',
 'SLC',
 'SNA',
 'SND',
 'SPM',
 'WAS']

In [23]:
fig = px.line(arrests_by_fy, x=fy, 
              y=aor, 
              title = "Arrests in AOR per FY",
              labels=dict(x="Fiscal Year", y="Number of Arrests"))
fig.update_xaxes(title="Fiscal Year", nticks = 4)
fig.update_yaxes(title="Number of Arrests")
fig.update_layout(legend_title_text='AOR')
fig.show()


In [13]:
import plotly.graph_objects as go

In [55]:
df = temo

In [60]:
df = arrests_by_fy.T.sort_values(
    by=["2018-10-01 00:00:00"], ascending=False)

cells=dict(values=df.transpose().values.tolist())
cells['values'].insert(0, temo.index.transpose().values.tolist())
cells['fill_color']='lavender'
cells['align']='left'

values = list(df.columns)
values.insert(0,'aor')
fig = go.Figure(data=[go.Table(
    header=dict(values=values,
                fill_color='paleturquoise',
                align='left'),
    cells= cells)
])

fig.show()

In [48]:
cells=dict(values=df.transpose().values.tolist())
cells['values'].insert(0, temo.index.transpose().values.tolist())
cells['fill_color']='lavender'
cells['align']='left'

In [54]:
cells

{'values': [['DAL',
   'ATL',
   'SNA',
   'HOU',
   'NOL',
   'MIA',
   'CHI',
   'LOS',
   'PHO',
   'SLC',
   'SFR',
   'SPM',
   'PHI',
   'WAS',
   'DET',
   'NEW',
   'SEA',
   'NYC',
   'BOS',
   'DEN',
   'SND',
   'ELP',
   'BAL',
   'BUF',
   'HQ'],
  [9634,
   8866,
   8425,
   12896,
   5174,
   3524,
   7055,
   7651,
   5370,
   4638,
   6651,
   2500,
   3672,
   2883,
   2241,
   2247,
   2698,
   1847,
   1858,
   2284,
   3683,
   1611,
   1239,
   1173,
   12],
  [16520,
   13551,
   8510,
   13565,
   7968,
   6192,
   8604,
   8419,
   6457,
   5177,
   7231,
   4175,
   4938,
   4163,
   3409,
   3189,
   3376,
   2576,
   2834,
   2746,
   4551,
   1892,
   1666,
   1494,
   20],
  [17644,
   15189,
   10749,
   14333,
   10270,
   8474,
   9241,
   7854,
   7162,
   5754,
   6210,
   4658,
   5143,
   4359,
   3604,
   3409,
   3067,
   3476,
   2908,
   2755,
   6000,
   2159,
   1703,
   1582,
   5],
  [15571,
   12268,
   10968,
   10761,
   9075,
   8695,
  

In [52]:
temo.index.transpose().values.tolist()

['DAL',
 'ATL',
 'SNA',
 'HOU',
 'NOL',
 'MIA',
 'CHI',
 'LOS',
 'PHO',
 'SLC',
 'SFR',
 'SPM',
 'PHI',
 'WAS',
 'DET',
 'NEW',
 'SEA',
 'NYC',
 'BOS',
 'DEN',
 'SND',
 'ELP',
 'BAL',
 'BUF',
 'HQ']

In [42]:
fig = go.Figure(data=[temo])

ValueError: 
    Invalid element(s) received for the 'data' property of 
        Invalid elements include: [apprehension_date  2015-10-01  2016-10-01  2017-10-01  2018-10-01
aor                                                              
DAL                      9634       16520       17644       15571
ATL                      8866       13551       15189       12268
SNA                      8425        8510       10749       10968
HOU                     12896       13565       14333       10761
NOL                      5174        7968       10270        9075
MIA                      3524        6192        8474        8695
CHI                      7055        8604        9241        7769
LOS                      7651        8419        7854        6144
PHO                      5370        6457        7162        5420
SLC                      4638        5177        5754        5014
SFR                      6651        7231        6210        4724
SPM                      2500     

In [14]:
fig = go.Figure(data=[go.Table(header=dict(values=['A Scores', 'B Scores']),
                 cells=dict(values=[[100, 90, 80, 90], [95, 85, 75, 95]]))
                     ])
fig.show()

In [30]:

# create transposed table of arrests over time in each AOR
temp = arrests_by_fy.T.sort_values(
    by=["2018-10-01 00:00:00"], ascending=False).style.format("{:,.0f}")

In [35]:
arrests_by_fy

aor,ATL,BAL,BOS,BUF,CHI,DAL,DEN,DET,ELP,HOU,...,NYC,PHI,PHO,SEA,SFR,SLC,SNA,SND,SPM,WAS
apprehension_date,,,,,,,,,,,,,,,,,,,,,
2015-10-01,8866,1239,1858,1173,7055,9634,2284,2241,1611,12896,...,1847,3672,5370,2698,6651,4638,8425,3683,2500,2883
2016-10-01,13551,1666,2834,1494,8604,16520,2746,3409,1892,13565,...,2576,4938,6457,3376,7231,5177,8510,4551,4175,4163
2017-10-01,15189,1703,2908,1582,9241,17644,2755,3604,2159,14333,...,3476,5143,7162,3067,6210,5754,10749,6000,4658,4359
2018-10-01,12268,1330,2266,1206,7769,15571,2215,3094,2103,10761,...,2281,3752,5420,2328,4724,5014,10968,2107,4005,3734


In [37]:
temo = arrests_by_fy.T.sort_values(
    by=["2018-10-01 00:00:00"], ascending=False)

In [41]:
temp

apprehension_date,2015-10-01 00:00:00,2016-10-01 00:00:00,2017-10-01 00:00:00,2018-10-01 00:00:00
aor,,,,
DAL,"9,634","16,520","17,644","15,571"
ATL,"8,866","13,551","15,189","12,268"
SNA,"8,425","8,510","10,749","10,968"
HOU,"12,896","13,565","14,333","10,761"
NOL,"5,174","7,968","10,270","9,075"
MIA,"3,524","6,192","8,474","8,695"
CHI,"7,055","8,604","9,241","7,769"
LOS,"7,651","8,419","7,854","6,144"
PHO,"5,370","6,457","7,162","5,420"


In [16]:
import plotly.express as px
df = px.data.iris()
fig = px.scatter(df, x="sepal_width", y="sepal_length", color="species")
fig.show()

In [17]:
fig.write_image('test.png')

In [18]:
!pip install -U kaleido

In [19]:
df = px.data.election()
geojson = px.data.election_geojson()
candidates = df.winner.unique()

In [22]:
candidates

array(['Joly', 'Coderre', 'Bergeron'], dtype=object)